### 데이터 값 목록 전체

* 데이터 값 전체 자료를 csv 파일로 저장

In [2]:
import os
import pandas as pd

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/"
file_list = os.listdir(path_dir)

df_all = pd.DataFrame()    
for file_name in file_list:
    if file_name != 'items_all.csv' and not file_name.startswith('__'):
        df = pd.read_csv(path_dir + file_name, dtype=object)
        df_all = pd.concat([df_all, df])
    
df_all.drop_duplicates(subset=['corp_code','rpt_num'], keep='first', inplace=True, ignore_index=True)            
df_all.to_csv(path_dir + 'items_all.csv', index=False)
len(df_all['corp_code'].unique())

340